In [ ]:
%cd ./yolov5
!python detect.py --source ../VidDemo.mp4 --weights ../models/best.pt --data ../fire_config.yaml --midas-model 0  --conf 0.29 --view-img  --half --vid-stride 2 > output.txt

In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog

class ImageConfigGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Configuration")
        
        self.reference_points = []
        self.email_address = ""
        
        # Create the buttons and labels
        self.open_btn = tk.Button(self.root, text="Open Configuration", command=self.open_configuration)
        self.open_btn.pack(pady=10)
        
        self.save_btn = tk.Button(self.root, text="Save Configuration", command=self.save_configuration)
        self.save_btn.pack(pady=10)
        
        self.reference_label = None  # Initialize reference_label as None


    def open_configuration(self):
            filename = filedialog.askopenfilename(filetypes=[("Text Files", "*.txt")])
            if filename:
                self.current_filename = filename 
                with open(filename, "r") as file:
                    lines = file.readlines()

                # Clear existing data
                self.reference_points = []
                self.email_addresses = []
                
                # Process each line and extract the data
                is_reference_points = False
                is_email_addresses = False

                for line in lines:
                    line = line.strip()

                    if line == "Reference Points:":
                        is_reference_points = True
                        is_email_addresses = False
                    elif line == "Email Address:":
                        is_reference_points = False
                        is_email_addresses = True
                    elif is_reference_points:
                        components = line.split(",")
                        if len(components) == 3:
                            x = int(components[0].strip())
                            y = int(components[1].strip())
                            z = float(components[2].strip())
                            self.reference_points.append([x, y, z])
                    elif is_email_addresses:
                        self.email_addresses.append(line)

                # Create a new window to display the points and email address
                config_window = tk.Toplevel(self.root)
                config_window.title("Configuration")

                # Create a frame for the reference points
                reference_frame = tk.Frame(config_window)
                reference_frame.pack(side=tk.LEFT, padx=10, pady=10)
                    
                # Create a label for the reference points count
                reference_count = len(self.reference_points)
                global refrence_label
                self.reference_label = tk.Label(reference_frame, text=f"Reference Points ({reference_count})")
                self.reference_label.pack()
                print(reference_label)
                # Create a listbox to display the reference points
                reference_listbox = tk.Listbox(reference_frame, height=10, width=30)
                reference_listbox.pack()

                # Populate the listbox with the reference points
                for point in self.reference_points:
                    reference_listbox.insert(tk.END, f"{point[0]}, {point[1]}, {point[2]}")

                # Create buttons for modifying and deleting reference points
                modify_btn = tk.Button(reference_frame, text="Modify", command=lambda: self.modify_reference_point(reference_listbox))
                modify_btn.pack(pady=5)

                delete_btn = tk.Button(reference_frame, text="Delete", command=lambda: self.delete_reference_point(reference_listbox))
                delete_btn.pack(pady=5)

                add_btn = tk.Button(reference_frame, text="Add", command=lambda: self.add_reference_point(reference_listbox))
                add_btn.pack(pady=5)

                # Create a frame for the email address
                email_frame = tk.Frame(config_window)
                email_frame.pack(side=tk.LEFT, padx=10, pady=10)

                # Create a label for the email address
                email_label = tk.Label(email_frame, text="Email Address")
                email_label.pack()

                # Create an entry field to display and modify the email address
                email_entry = tk.Entry(email_frame)
                email_entry.pack()

                # Set the initial value of the email address if it exists
                if self.email_addresses:
                    email_entry.insert(tk.END, self.email_addresses[0])

                # Create a button to update the email address
                update_btn = tk.Button(email_frame, text="Update", command=lambda: self.update_email_address(email_entry))
                update_btn.pack(pady=5)

                # Create a button to save the data in the same file
                save_btn = tk.Button(config_window, text="Save Configuration", command=self.save_configuration)
                save_btn.pack(pady=5)
                

    def modify_reference_point(self, listbox):
        selection = listbox.curselection()
        if selection:
            point = self.reference_points[selection[0]]
            point_str = f"{point[0]}, {point[1]}, {point[2]}"
            new_point_str = simpledialog.askstring("Modify Reference Point", "Enter the modified reference point:", initialvalue=point_str)
            if new_point_str:
                components = new_point_str.split(",")
                if len(components) == 3:
                    x = int(components[0].strip())
                    y = int(components[1].strip())
                    z = float(components[2].strip())
                    self.reference_points[selection[0]] = [x, y, z]
                    listbox.delete(selection[0])
                    listbox.insert(selection[0], new_point_str)
             # Update the label showing the number of reference points
            self.update_reference_label( listbox)

    def update_reference_label(self, listbox):
        reference_count = listbox.size()
        print(reference_count)
        if self.reference_label:
            self.reference_label.config(text=f"Reference Points ({reference_count})")


    def delete_reference_point(self, listbox):
        selection = listbox.curselection()
        if selection:
            self.reference_points.pop(selection[0])
            listbox.delete(selection[0])
            self.update_reference_label( listbox)
        
    
    def add_reference_point(self, listbox):
        new_point_str = simpledialog.askstring("Add Reference Point", "Enter the new reference point:")
        if new_point_str:
            components = new_point_str.split(",")
            if len(components) == 3:
                x = int(components[0].strip())
                y = int(components[1].strip())
                z = float(components[2].strip())
                self.reference_points.append([x, y, z])
                listbox.insert(tk.END, new_point_str)
                self.update_reference_label(listbox)

            
    def update_email_address(self, email_entry):
        email = email_entry.get()
        if email:
            self.email_addresses = [email]
            messagebox.showinfo("Information", "Email address updated successfully.")
        else:
            messagebox.showwarning("Warning", "Please enter a valid email address.")
        
    def save_configuration(self):
        filename = self.current_filename  # Use the previously opened file name
        if filename:
            with open(filename, "w") as file:
                file.write("Reference Points:\n")
                for point in self.reference_points:
                    file.write(f"{point[0]}, {point[1]}, {point[2]}\n")

                file.write("\nEmail Address:\n")
                for email in self.email_addresses:
                    file.write(f"{email}\n")

            messagebox.showinfo("Information", "Configuration saved successfully.")
        else:
            messagebox.showwarning("Warning", "No file is currently open.")
            
# Create the main window
root = tk.Tk()

# Create the GUI object
gui = ImageConfigGUI(root)

# Start the GUI event loop
root.mainloop()
